In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import cv2
import json
import torch
import zipfile
import numpy as np
import pandas as pd
import plotly.express as px

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")

In [ ]:
from params import *

from inference.main import k_fold_inf

from data.dataset import InferenceDataset
from data.transforms import HE_preprocess

### Load

In [ ]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")
df = pd.read_csv(OUT_PATH + "df_images.csv")

### Data

In [ ]:
root = TIFF_PATH_4
rle_path = DATA_PATH + "train_4.csv"
reduce_factor = 1
rles = pd.read_csv(rle_path)

In [ ]:
img = "2f6ecfcdf" # "aaa6a05cc"

In [ ]:
dataset = InferenceDataset(
    f"{root}/{img}.tiff",
    rle=rles[rles['id'] == img]["encoding"],
    overlap_factor=2,
    reduce_factor=reduce_factor,
    transforms=HE_preprocess(augment=False, visualize=True),
)

In [ ]:
img, pos = dataset[1000]

plt.imshow(img.numpy().transpose(1, 2, 0))

### Inference

In [ ]:
log_folder = "../logs/2021-03-18/0/"  # b5 new

In [ ]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [ ]:
global_threshold = 0.4

In [ ]:
%%time
scores = k_fold_inf(
    config,
    df,
    log_folder=log_folder,
    use_full_size=False,
    global_threshold=global_threshold,
    use_tta=False,
    save=True,
)

In [ ]:
print(f'Local CV score is {np.mean(scores):.4f} for threshold {global_threshold}')